## Set-up

Necessary imports and helper functions for displaying points, boxes, and masks.

In [ ]:
%cd '/home/xiaobo/Project/Trans-WSSS'
%pwd

import sys
sys.path.append("others/segment_anything")

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    


## Example image

In [ ]:
# image = cv2.imread('others/segment-anything/notebooks/images/truck.jpg')
image = cv2.imread('datasets/VOC2012/JPEGImages/2007_000480.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.axis('on')
plt.show()

## Selecting objects with SAM

First, load the SAM model and predictor. Change the path below to point to the SAM checkpoint. Running on CUDA and using the default model are recommended for best results.

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "pretrains/SAM/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda:0"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

Process the image to produce an image embedding by calling `SamPredictor.set_image`. `SamPredictor` remembers this embedding and will use it for subsequent mask prediction.

In [ ]:
predictor.set_image(image)
low_res_pad_h, low_res_pad_w = (256 - predictor.input_size[0] // 4), (256 - predictor.input_size[1] // 4)
print(f"low_res_pad_h: {low_res_pad_h}, low_res_pad_w: {low_res_pad_w}")

To select the truck, choose a point on it. Points are input to the model in (x,y) format and come with labels 1 (foreground point) or 0 (background point). Multiple points can be input; here we use only one. The chosen point will be shown as a star on the image.

In [ ]:
input_point = np.array([[48, 226]])
input_label = np.array([1])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()  

Predict with `SamPredictor.predict`. The model returns masks, quality predictions for those masks, and low resolution mask logits that can be passed to the next iteration of prediction.

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)
multi_logits = logits

In [ ]:
mask_logits, _, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
    return_logits=True
)

With `multimask_output=True` (the default setting), SAM outputs 3 masks, where `scores` gives the model's own estimation of the quality of these masks. This setting is intended for ambiguous input prompts, and helps the model disambiguate different objects consistent with the prompt. When `False`, it will return a single mask. For ambiguous prompts such as a single point, it is recommended to use `multimask_output=True` even if only a single mask is desired; the best single mask can be chosen by picking the one with the highest score returned in `scores`. This will often result in a better mask.

In [ ]:
print(masks.shape)  # (number_of_masks) x H x W
print(scores.shape)
print(logits.shape)

In [ ]:
print(f"{logits.mean()=}, {logits.std()=}, {logits.min()=}, {logits.max()=}")
logits[0]

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(logits.transpose(1, 0, 2).reshape(256, -1))
plt.colorbar(fraction=0.02)
plt.axis('on')
plt.show()

In [ ]:
for i, logit in enumerate(logits):
    print(f"mask {i} padding logit: {logit[-low_res_pad_h:, -low_res_pad_w:].mean()=}")

for i, logit in enumerate(mask_logits):
    print(f"mask {i} fg logit: {logit[masks[i]].mean()=}")
    print(f"mask {i} bg logit: {logit[~masks[i]].mean()=}")

In [ ]:
plt.figure(figsize=(30, 10))
for i, logit in enumerate(logits):
    plt.subplot(1, 3, i + 1)
    plt.hist(logit.flatten(), bins=50)
    plt.yscale('log', base=10)
    plt.tick_params(axis='both', which='major', labelsize=20)
plt.show()

In [ ]:
print(mask_logits.shape)  # (number_of_masks) x H x W
plt.figure(figsize=(20, 20))
plt.imshow(mask_logits.transpose(1, 0, 2).reshape(image.shape[0], -1))
plt.colorbar(fraction=0.01)
plt.axis('on')
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.subplot(1, 3, i+1)
    plt.imshow(image)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')

plt.show()

## Specifying a specific object with additional points

The single input point is ambiguous, and the model has returned multiple objects consistent with it. To obtain a single object, multiple points can be provided. If available, a mask from a previous iteration can also be supplied to the model to aid in prediction. When specifying a single object with multiple prompts, a single mask can be requested by setting `multimask_output=False`.

In [ ]:
input_point = np.array([[48, 226], [182, 237]])
input_label = np.array([1, 1])

mask_input = multi_logits[np.argmax(scores), :, :]  # Choose the model's best mask

In [ ]:
masks, _, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)

In [ ]:
mask_logits, _, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
    return_logits=True
)

In [ ]:
print(masks.shape)  # (number_of_masks) x H x W
print(scores.shape)
print(logits.shape)

In [ ]:
print(f"{logits.mean()=}, {logits.std()=}, {logits.min()=}, {logits.max()=}")
logits[0]

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(logits.transpose(1, 0, 2).reshape(256, -1))
plt.colorbar(fraction=0.1)
plt.axis('on')
plt.show()

In [ ]:
for i, logit in enumerate(logits):
    print(f"mask {i} padding logit: {logit[-low_res_pad_h:, -low_res_pad_w:].mean()=}")

for i, logit in enumerate(mask_logits):
    print(f"mask {i} fg logit: {logit[masks[i]].mean()=}")
    print(f"mask {i} bg logit: {logit[~masks[i]].mean()=}")

In [ ]:
plt.figure(figsize=(10, 10))
for i, logit in enumerate(logits):
    plt.subplot(1, 1, i + 1)
    plt.hist(logit.flatten(), bins=50)
    plt.yscale('log', base=10)
    plt.tick_params(axis='both', which='major', labelsize=20)
plt.show()

In [ ]:
print(mask_logits.shape)  # (number_of_masks) x H x W
plt.figure(figsize=(10, 10))
plt.imshow(mask_logits.transpose(1, 0, 2).reshape(image.shape[0], -1))
plt.colorbar(fraction=0.025)
plt.axis('on')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)
show_mask(masks, plt.gca())
show_points(input_point, input_label, plt.gca())
plt.axis('off')
plt.show() 

To exclude the car and specify just the window, a background point (with label 0, here shown in red) can be supplied.

In [ ]:
input_point = np.array([[48, 226], [182, 237], [267, 345]])
input_label = np.array([1, 1, 0])

mask_input = multi_logits[np.argmax(scores), :, :]  # Choose the model's best mask

In [ ]:
masks, _, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)

In [ ]:
mask_logits, _, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
    return_logits=True
)

In [ ]:
print(masks.shape)  # (number_of_masks) x H x W
print(scores.shape)
print(logits.shape)

In [ ]:
print(f"{logits.mean()=}, {logits.std()=}, {logits.min()=}, {logits.max()=}")
logits[0]

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(logits.transpose(1, 0, 2).reshape(256, -1))
plt.colorbar(fraction=0.1)
plt.axis('on')
plt.show()

In [ ]:
for i, logit in enumerate(logits):
    print(f"mask {i} padding logit: {logit[-low_res_pad_h:, -low_res_pad_w:].mean()=}")

for i, logit in enumerate(mask_logits):
    print(f"mask {i} fg logit: {logit[masks[i]].mean()=}")
    print(f"mask {i} bg logit: {logit[~masks[i]].mean()=}")

In [ ]:
plt.figure(figsize=(10, 10))
for i, logit in enumerate(logits):
    plt.subplot(1, 1, i + 1)
    plt.hist(logit.flatten(), bins=50)
    plt.yscale('log', base=10)
    plt.tick_params(axis='both', which='major', labelsize=20)
plt.show()

In [ ]:
print(mask_logits.shape)  # (number_of_masks) x H x W
plt.figure(figsize=(10, 10))
plt.imshow(mask_logits.transpose(1, 0, 2).reshape(image.shape[0], -1))
plt.colorbar(fraction=0.025)
plt.axis('on')
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
show_mask(masks, plt.gca())
show_points(input_point, input_label, plt.gca())
plt.axis('off')
plt.show()

In [ ]:
# Not Used